# Analysing COD/CIS Images

## Values in "unknown" part(s) of header

In [ ]:
from dsat import Dsat
import pandas as pd

d = Dsat.from_file("../dsatnord.mp")

data = []
for t in d.tiles_zoom2.tiles:
    data.append([int(h) for h in t.header.unknown5])

df = pd.DataFrame(data)
df.nunique().plot.bar()

## Stats about "cod" parts

### Number of parts per tile

In [ ]:
import pandas as pd
from cis import Cis

def le16(b):
    return int.from_bytes(b, byteorder="little", signed=False)

def get_planes(b):
    """Statistics about "cod" parts in byte array b."""
    planes = []
    lastpos = 0
    for i in range(0, len(b) - 2):
        if chr(b[i]) == 'c' and chr(b[i+1]) == 'o' and chr(b[i+2]) == 'd':
            planes.append(b[lastpos:i])
            lastpos = i + 3
    return planes

img = "../img/wolf.cod"
img = "../img/tile_644684211.cod"

c = Cis.from_file(img)

planes = get_planes(c.data.planes)

[(le16(b[0:2]), le16(b[2:4]), le16(b[4:6]), len(b)) for b in planes]

In [ ]:
from dsat import Dsat
import pandas as pd

d = Dsat.from_file("../dsatnord.mp")

def count_cod(b):
    """Statistics about "cod" parts in byte array b."""
    pos = []
    for i in range(0, len(b) - 2):
        if chr(b[i]) == 'c' and chr(b[i+1]) == 'o' and chr(b[i+2]) == 'd':
            pos.append(i)
    return len(pos)

print(0, pd.DataFrame([count_cod(t.data.unknown) for t in d.tiles_zoom0.tiles]).value_counts(), sep='\n')
print(1, pd.DataFrame([count_cod(t.data.unknown) for t in d.tiles_zoom1.tiles]).value_counts(), sep='\n')
print(2, pd.DataFrame([count_cod(t.data.unknown) for t in d.tiles_zoom2.tiles]).value_counts(), sep='\n')
print(3, pd.DataFrame([count_cod(t.data.unknown) for t in d.tiles_zoom3.tiles]).value_counts(), sep='\n')

### Number of distinct distances between parts

In [ ]:
from dsat import Dsat
import pandas as pd

d = Dsat.from_file("../dsatnord.mp")

def count_cod(b):
    """Statistics about "cod" parts in byte array b."""
    dists = set()
    lastpos = 0
    for i in range(0, len(b) - 2):
        if chr(b[i]) == 'c' and chr(b[i+1]) == 'o' and chr(b[i+2]) == 'd':
            dists.add(i - lastpos)
    return len(dists)

print(0, pd.DataFrame([count_cod(t.data.unknown) for t in d.tiles_zoom0.tiles]).value_counts(), sep='\n')
print(1, pd.DataFrame([count_cod(t.data.unknown) for t in d.tiles_zoom1.tiles]).value_counts(), sep='\n')
print(2, pd.DataFrame([count_cod(t.data.unknown) for t in d.tiles_zoom2.tiles]).value_counts(), sep='\n')
print(3, pd.DataFrame([count_cod(t.data.unknown) for t in d.tiles_zoom3.tiles]).value_counts(), sep='\n')

## Size Distribution of Sub-Bands

In [ ]:
from dsat import Dsat
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['figure.dpi'] = 140

def cod_sizes(b):
    """Sizes of "cod"-delimited parts in byte array b."""
    sizes = []
    lastpos = 0
    for i in range(0, len(b) - 2):
        if chr(b[i]) == 'c' and chr(b[i+1]) == 'o' and chr(b[i+2]) == 'd':
            sizes.append(i - lastpos)
    return sizes

df = pd.DataFrame([cod_sizes(t.data.planes) for t in Dsat.from_file("../dsatnord.mp").tiles_zoom3.tiles])

ax = sns.violinplot(df, inner=None, split=False, log_scale=True, zorder=3, density_norm="count", common_norm=True)
plt.ylabel("size [bytes]")
plt.xlabel("sub-band")
plt.grid(axis="y", which="both", zorder=0)
for i in df.columns:
    ax.text(i + 0.1, df[[i]].min().iloc[0], "{:2.0f}".format(df[[i]].mean().iloc[0]))

plt.show()

In [ ]:
df[[1]].max()